In [1]:
from pathlib import Path

funcs_csv = Path('/home/cls0027/exp_builds/astera.exp/rundata/run1/6.fighter/functions.csv')
locals_csv = Path('/home/cls0027/exp_builds/astera.exp/rundata/run1/6.fighter/locals.csv')

import pandas as pd
pd.set_option('display.max_colwidth', 20)   # JSON columns make things look weird in notebook without this


funcs_df = pd.read_csv(funcs_csv)
locals_df = pd.read_csv(locals_csv)

In [2]:
from astlib import StructDatabase
#!ls ~/exp_builds/astera.exp/rundata/run1/0.fighter/0.fighter.debug.sdb

debug_sdb_file = Path.home()/'exp_builds/astera.exp/rundata/run1/6.fighter/6.fighter.debug.sdb'
strip_sdb_file = Path.home()/'exp_builds/astera.exp/rundata/run1/6.fighter/6.fighter.sdb'
sdb = StructDatabase.from_json(debug_sdb_file)
strip_sdb = StructDatabase.from_json(strip_sdb_file)

In [3]:
sid = sdb.sids_by_name['r_camera'][0]
sdb.structs_by_id[sid]

struct r_camera {
	0x0: float[3] position
	0xc: float[2] size
	0x14: float rotation
	0x18: float[4][4] view
	0x58: float[4][4] projection
	0x98: float near
	0x9c: float far
}

In [4]:
locals_df.loc[locals_df.TypeSeq_Debug.apply(lambda x: 'STRUCT' in x)].groupby('TypeSeq_Debug').count()

,FunctionStart,Signature,Name_Strip,Type_Strip,LocType_Strip,LocRegName_Strip,LocOffset_Strip,TypeCategory_Strip,TypeSeq_Strip,Name_Debug,Type_Debug,LocType_Debug,LocRegName_Debug,LocOffset_Debug,TypeCategory_Debug,HasDWARF,TypeJson_Debug,BinaryId
TypeSeq_Debug,,,,,,,,,,,,,,,,,,
"ARR,STRUCT",4,4,4,4,4,0,4,4,4,4,4,4,0,4,4,4,4,4
"PTR,PTR,STRUCT",10,10,10,10,10,4,10,10,10,10,10,10,4,10,10,10,10,10
"PTR,STRUCT",444,444,444,444,444,242,444,444,444,444,444,444,242,444,444,444,444,444
STRUCT,59,59,59,59,59,0,59,59,59,59,59,59,0,59,59,59,59,59


In [5]:
# start with PTR->STRUCT, this is likely the easiest

var_row = locals_df[locals_df.TypeSeq_Debug=='PTR,STRUCT'].iloc[0]
var_row

FunctionStart                     1090825
Signature                        27,64,89
Name_Strip                          lVar2
Type_Strip                          int64
LocType_Strip                    register
LocRegName_Strip                      rax
LocOffset_Strip                         0
TypeCategory_Strip                BUILTIN
TypeSeq_Strip                       int64
Name_Debug                          asset
Type_Debug                       asset_t*
LocType_Debug                    register
LocRegName_Debug                      rax
LocOffset_Debug                       0.0
TypeCategory_Debug                    PTR
TypeSeq_Debug                  PTR,STRUCT
HasDWARF                            False
TypeJson_Debug        {"kind": "Pointe...
BinaryId                                6
Name: 15, dtype: object

Read datatype from JSON, since we have the struct db we have the layout:

In [6]:
import astlib
from varlib.datatype import datatype_from_json_str

dt = datatype_from_json_str(var_row.TypeJson_Debug, sdb)
dt.pointed_to.layout

0x0: uint32 uid
0x8: uchar* data
0x10: uint32 data_length
0x18: char* name
0x20: int32 chunk_start
0x24: int32 chunk_length
0x28: uchar fs
0x29: uchar filled
0x2a: uchar req
0x2b: uchar req_free
0x2c: uchar chunk

Ok, so let's collect all the member accesses from the Debug AST for this variable

In [7]:
var_name = var_row.Name_Debug
strip_name = var_row.Name_Strip
func_addr = var_row.FunctionStart
bin_id = var_row.BinaryId
print(f'Binary: {bin_id}, func {func_addr:#x}, local debug var "{var_name}", local strip var  "{strip_name}"')

Binary: 6, func 0x10a509, local debug var "asset", local strip var  "lVar2"


In [8]:
func_row = funcs_df[funcs_df.FunctionStart==func_addr].iloc[0]
debug_ast_jsonfile = func_row.AstJson_Debug
strip_ast_jsonfile = func_row.AstJson_Strip
# print(debug_ast_jsonfile)
debug_ast = astlib.read_json(debug_ast_jsonfile, sdb)
strip_ast = astlib.read_json(strip_ast_jsonfile, strip_sdb)

In [9]:
# --------------------------------------------------
# TODO: pick up here and VISIT the AST to collect all MemberExpr
# nodes that involve our target variable
# --------------------------------------------------
# --> or collect ALL MemberExprs and try to map each of them to StripAST

# NOTE: see initial start on implementation in basic_dataset for reference...

vdecl = debug_ast.inner[-1].inner[-1].inner[1].inner[0]
vdecl.dtype.pointed_to

struct asset_t {
	0x0: uint32 uid
	0x8: uchar* data
	0x10: uint32 data_length
	0x18: char* name
	0x20: int32 chunk_start
	0x24: int32 chunk_length
	0x28: uchar fs
	0x29: uchar filled
	0x2a: uchar req
	0x2b: uchar req_free
	0x2c: uchar chunk
}

In [10]:
from typing import Any, Callable, List
from astlib import *

class CollectStructMemberRefs(VisitAllChildrenByDefaultVisitor):
    def __init__(self):
        '''
        ast: The AST to search for MemberExpr nodes
        '''
        super().__init__()
        # self.member_exprs:List[MemberExpr] = []

    def visit_MemberExpr(self, memexpr:MemberExpr):
        # self.member_exprs.append(memexpr)
        return memexpr

memberexprs = CollectStructMemberRefs().visit(debug_ast)
memberexprs

[<DeclRef: asset>->data (offset=0x8, sid=72057594037927950),
 <DeclRef: asset_00>->data (offset=0x8, sid=72057594037927950)]

In [11]:
m:MemberExpr = memberexprs[0]
m.parent_struct
# m.offset
# m.parent_struct.layout.fields_by_offset[m.offset].dtype
print(m, hex(m.instr_addr))

<DeclRef: asset>->data (offset=0x8, sid=72057594037927950) 0x10a549


In [12]:
matching_int_literals = [x for x in strip_ast.nodes_at_addr(m.instr_addr) if isinstance(x, IntegerLiteral)]
if matching_int_literals:
    print(matching_int_literals[0])

<IntegerLiteral: 8 (dtype=int64)>


## Pick up here:
1. What do we do with >1 IntegerLiteral match?
2. What do we do if there are no IntegerLiteral matches? (*this is probably fine, but for now stop and look at why*)
3. Are there other "artifacts" of member references in the stripped AST besides IntegerLiteral nodes?
    > - Look specifically at stack-based structs...I don't anticipate there is anything "direct"...we may just have to
    **detect non-stack-based structs** first, then "find" the stack-based ones via type propagation/decompiler analysis (even if we help Ghidra finish this...)